In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Custom DecisionTree
## Classes and functions

In [2]:
def transform_data(df):
    """ Transform the dataset into a list of rows. """
    return [row for row in df.itertuples(index=False, name='Pandas')]


def is_numeric(value):
    """ Check if the values is numeric. """
    return isinstance(value, int) or isinstance(value, float)


def gini(rows):
    """
    Calculate the Gini Impurity for a list of rows (загрязненность; примеси).
    Source: https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = label_counts(rows)
    #print(counts)
    impurity = 1
    for target in counts:
        #print(target)
        p_i = counts[target] / float(len(rows))
        #print(p_i)
        impurity -= p_i ** 2
    return impurity


def info_gain(left, right, current_uncertainty):
    """
    Information Gain.
    The uncertainty of the starting node, minus the weighted impurity of two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)


def label_counts(rows):
    """
    Counts the number of each possible value of example (label) in a dataset (for target feature).
    Besides, for predictions counts the number of times a particular label appeared in a leaf node.
    """
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # target value is considered to be the last column in the dataset
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [3]:
class Question:
    """
    For each Node in a Tree there is a Question. Records a column number and a column value.
    The 'match' method is used to compare the feature value in an example (row) to the feature value (& condition) stored in the Question.
    """
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the feature value in this question.
        # Returns True or False.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # print the question in a readable form;
        # purely for beautiful output
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            headers[self.column], condition, str(self.value))

In [4]:
class Leaf:
    """
    A Leaf node.
    Holds a dictionary of class -> number of times it appears in the rows from the train set that reaches this leaf.
    """
    def __init__(self, rows):
        self.predictions = label_counts(rows)

In [5]:
class Decision_Node:
    """
    A Decision Node has a question.
    Holds a reference to the question, and to the two child nodes.
    """
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

    def print_tree(self, node, spacing=""):

        # Base case: we've reached a leaf
        if isinstance(node, Leaf):
            print (spacing + "PREDICT", node.predictions)
            return

        # print the question at this node
        print (spacing + str(node.question))

        # Call this function recursively on the 'True branch'
        print (spacing + '--> True:')
        self.print_tree(node.true_branch, spacing + "  ")

        # Call this function recursively on the 'False branch'
        print (spacing + '--> False:')
        self.print_tree(node.false_branch, spacing + "  ")

## Tree building

In [6]:
class DecisionTree:
    
    def __init__(self, max_depth=np.inf):
        self.max_depth = max_depth
        self.initial_depth = 0
        #self.root = None


    def partition(self, rows, question):
        """
        Partitions a dataset into 'True rows' and 'False rows'.
        For each row in the dataset, check if it matches the question (its condition).
        If so, add it to 'True rows', otherwise, add it to 'False rows'.
        """
        true_rows, false_rows = [], []
        for row in rows:
            if question.match(row):
                true_rows.append(row)
            else:
                false_rows.append(row)
        return true_rows, false_rows


    def find_best_split(self, rows):
        """
        Find the best question by iterating over every feature/value (condition) and calculating the information gain.
        """
        best_gain = 0  # best info gain (initial)
        best_question = None  # the feature / value that produced the gain (initial)
        current_uncertainty = gini(rows)  # initial uncertainty
        n_feats = len(rows[0]) - 1  # number of columns (exclude the target)

        for col in range(n_feats):  # for each feature

            values = set([row[col] for row in rows])  # unique values in the column

            for val in values:  # for each value in the unique set

                question = Question(col, val)  # current feature/value (condition)

                # try splitting the dataset
                true_rows, false_rows = self.partition(rows, question)

                # skip this split if it doesn't divide the dataset
                if len(true_rows) == 0 or len(false_rows) == 0:
                    continue

                # calculate the information gain from this split
                gain = info_gain(true_rows, false_rows, current_uncertainty)

                # we can also use '>='
                if gain > best_gain:
                    best_gain, best_question = gain, question

        return best_gain, best_question


    def build_tree(self, rows):
        """ Builds a tree using recursion. """
        
        # increase the depth for the test above
        self.initial_depth += 1

        # check if max_depth is reached
        # if so, leave
        if self.initial_depth >= self.max_depth:
            return Leaf(rows)

        # get the best info gain and the question that produces it
        gain, question = self.find_best_split(rows)

        # base case: no futher info gain
        # if no futher questions => return a leaf
        if gain == 0:
            return Leaf(rows)

        # if we reach here, we have found a useful feature/value (condition) to partition on
        true_rows, false_rows = self.partition(rows, question)

        # recursively build the 'True branch'.
        true_branch = self.build_tree(true_rows)

        # recursively build the 'False branch'.
        false_branch = self.build_tree(false_rows)

        # Returns a Question node along with two branches.
        # Records the best feature/value (condition) to ask at this point,
        # as well as the branches to follow depending on the answer.
        return Decision_Node(question, true_branch, false_branch)


    def fit(self, training_data):
        """ Fits the training data by building a tree. """
        rows = transform_data(training_data)
        return self.build_tree(rows)

In [18]:
def classify(row, node):
    """ Makes a prediction for a row. """

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the 'True branch' or the 'False branch' using 'match' function.
    # Then classify the branches.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)


def predict(my_tree, testing_data):
    """ Returns predictions. """

    rows = transform_data(testing_data)
    
    # get a list of dictionaries
    pred_dicts = [classify(rows[i], my_tree) for i in range(len(rows))]
    
    # sort each dictionary by value and get a list of tuples
    tuples = [sorted(row.items(), reverse=True, key=lambda k: k[1]) for row in pred_dicts]
    
    # return a list of first elements in each tuple;
    # result: the list of most likely predictions 
    return [i[0][0] for i in tuples]


def score(actual, pred):
    """ Returns accuracy and indexes of rows with wrong predictions. """
    cnt = 0
    wrong_idxs = []
    for i in range(len(actual)):
        if pred[i] != actual[i]:
            cnt += 1
            wrong_idxs.append(i)
    return 1 - cnt / len(actual), wrong_idxs


def print_leaf(counts):
    """ Predictions with porbabilities. """
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(round(counts[lbl] / total * 100, 3)) + "%"
    return probs

# Testing the DecisionTree

## Iris dataset
https://en.wikipedia.org/wiki/Iris_flower_data_set
### Data

In [7]:
iris = sns.load_dataset('iris')
iris.head(3)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa


In [8]:
iris.tail(3)

,sepal_length,sepal_width,petal_length,petal_width,species
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


In [9]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal_length    150 non-null float64
sepal_width     150 non-null float64
petal_length    150 non-null float64
petal_width     150 non-null float64
species         150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


In [10]:
iris.isna().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [11]:
iris.species.unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [12]:
# split iris dataset
X_iris_train, X_iris_test, y_iris_train, y_iris_test = train_test_split(iris.drop('species', axis=1), iris.species)

In [13]:
X_iris_test.head(3)

,sepal_length,sepal_width,petal_length,petal_width
118,7.7,2.6,6.9,2.3
36,5.5,3.5,1.3,0.2
102,7.1,3.0,5.9,2.1


### Iris: predict and compare

In [14]:
X_iris_train['species'] = y_iris_train
X_iris_train.head(3)

,sepal_length,sepal_width,petal_length,petal_width,species
125,7.2,3.2,6.0,1.8,virginica
2,4.7,3.2,1.3,0.2,setosa
129,7.2,3.0,5.8,1.6,virginica


In [15]:
headers = list(iris.columns)
headers

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

In [16]:
my_tree = DecisionTree(max_depth=3)
fitted3 = my_tree.fit(X_iris_train)
fitted3.print_tree(fitted3)

Is petal_length >= 3.0?
--> True:
  Is petal_length >= 4.8?
  --> True:
    PREDICT {'virginica': 36, 'versicolor': 4}
  --> False:
    PREDICT {'versicolor': 34}
--> False:
  PREDICT {'setosa': 38}


In [19]:
predictions3 = predict(fitted3, X_iris_test)
predictions3

['virginica',
 'setosa',
 'virginica',
 'setosa',
 'versicolor',
 'setosa',
 'setosa',
 'setosa',
 'virginica',
 'setosa',
 'virginica',
 'versicolor',
 'virginica',
 'virginica',
 'virginica',
 'versicolor',
 'versicolor',
 'virginica',
 'versicolor',
 'versicolor',
 'setosa',
 'versicolor',
 'versicolor',
 'virginica',
 'setosa',
 'virginica',
 'virginica',
 'virginica',
 'virginica',
 'setosa',
 'virginica',
 'versicolor',
 'setosa',
 'virginica',
 'setosa',
 'setosa',
 'versicolor',
 'versicolor']

In [20]:
score(list(y_iris_test), predictions3)

(0.9210526315789473, [4, 12, 23])

### Iris: Real DT

In [21]:
dt = DecisionTreeClassifier(max_depth=3)
dt.fit(X_iris_train.drop('species', axis=1), y_iris_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [22]:
score(list(y_iris_test), dt.predict(X_iris_test))

(0.8947368421052632, [4, 12, 13, 23])

In [23]:
accuracy_score(y_iris_test, dt.predict(X_iris_test))

0.8947368421052632

## Churn dataset
### Data

In [24]:
df = pd.read_csv('data/telecom_churn.csv', sep=',')
df.drop('State', axis=1, inplace=True)
df.head(3)

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False


In [25]:
df.shape

(3333, 19)

In [26]:
df['International plan'] = df['International plan'].map(dict(Yes=1, No=0))
df['Voice mail plan'] = df['Voice mail plan'].map(dict(Yes=1, No=0))
df.head(3)

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,415,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,107,415,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False


In [27]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Churn', axis=1), df.Churn)

In [28]:
X_train.shape[0], X_test.shape[0], y_train.shape[0], y_test.shape[0]

(2499, 834, 2499, 834)

In [29]:
X_test.head(3)

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls
1746,60,408,0,0,0,179.3,147,30.48,208.9,89,17.76,248.2,98,11.17,13.5,6,3.65,1
2230,109,510,1,0,0,209.1,141,35.55,205.0,93,17.43,119.4,111,5.37,7.8,3,2.11,2
2977,132,408,0,0,0,195.1,100,33.17,148.8,95,12.65,224.5,117,10.10,6.7,2,1.81,0


### Churn: predict and compare

In [30]:
X_train['Churn'] = y_train

In [31]:
headers = list(df.columns)
headers

['Account length',
 'Area code',
 'International plan',
 'Voice mail plan',
 'Number vmail messages',
 'Total day minutes',
 'Total day calls',
 'Total day charge',
 'Total eve minutes',
 'Total eve calls',
 'Total eve charge',
 'Total night minutes',
 'Total night calls',
 'Total night charge',
 'Total intl minutes',
 'Total intl calls',
 'Total intl charge',
 'Customer service calls',
 'Churn']

In [32]:
%%time
my_tree = DecisionTree(max_depth=7)
fitted4 = my_tree.fit(X_train)
fitted4.print_tree(fitted4)

Is Total day minutes >= 263.7?
--> True:
  Is Voice mail plan >= 1?
  --> True:
    Is International plan >= 1?
    --> True:
      Is Account length >= 60?
      --> True:
        PREDICT {True: 3}
      --> False:
        PREDICT {False: 1}
    --> False:
      PREDICT {False: 38}
  --> False:
    PREDICT {True: 101, False: 24}
--> False:
  PREDICT {False: 2062, True: 270}
CPU times: user 24.9 s, sys: 0 ns, total: 24.9 s
Wall time: 24.9 s


In [33]:
predictions4 = predict(fitted4, X_test)
predictions4[:10]

[False, False, False, True, True, False, False, True, False, False]

In [34]:
score, idxs = score(list(y_test), predictions4)
score, len(idxs)

(0.8776978417266187, 102)

In [35]:
accuracy_score(y_test, predictions4)

0.8776978417266187

### Churn: real DT

In [36]:
dtt = DecisionTreeClassifier(max_depth=7)
dtt.fit(X_train.drop('Churn', axis=1), y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [37]:
accuracy_score(y_test, dtt.predict(X_test))

0.947242206235012